**carte des voies ferrées**

In [ ]:
import geopandas as gpd
import requests
import folium
from io import BytesIO

# URLs des fichiers GeoParquet
url_voies_ferrees = "https://github.com/Ghithub23/final/raw/main/voies_ferrees.parquet"

# Fonction pour charger un GeoParquet depuis une URL
def load_geoparquet(url):
    response = requests.get(url)
    response.raise_for_status()  # Vérifie que la réponse est valide
    return gpd.read_parquet(BytesIO(response.content))

# Charger les GeoParquets
gdf_voies_ferrees = load_geoparquet(url_voies_ferrees)

# Afficher les colonnes disponibles dans chaque GeoParquet
print("Colonnes dans le fichier 'voies_ferrees' :", gdf_voies_ferrees.columns)

# Mettre à jour les limites géographiques au Maroc
gdf_voies_ferrees = gdf_voies_ferrees.cx[-17.0:-1.0, 21.0:36.0]

# Create the Folium map object here before the loop
# Calculate center coordinates
center_coords = [
    gdf_voies_ferrees.geometry.centroid.y.mean(),  # Use gdf_voies_ferrees for center
    gdf_voies_ferrees.geometry.centroid.x.mean()
]
m = folium.Map(location=center_coords, zoom_start=6)

# Palette de couleurs pour les voies ferrées
railway_color_map = {
    "rail": "red",
    "tram": "brown"
}


# Ajouter les voies ferrées avec des couleurs spécifiques
for _, row in gdf_voies_ferrees.iterrows():
    # Récupérer la couleur basée sur 'fclass'
    color = railway_color_map.get(row['fclass'], "darkgray")  # Gris foncé par défaut si type inconnu
    folium.GeoJson(
        row.geometry.__geo_interface__,
        style_function=lambda feature, color=color: {
            'color': color,
            'weight': 2
        }
    ).add_to(m)

# Créer une légende combinée pour les bâtiments et les voies ferrées
legend_html = """
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 250px; height: auto;
    z-index: 1000; background-color: white;
    padding: 10px; border: 2px solid black; border-radius: 5px;
">
    <h4>Légende</h4>
    <h5>Voies Ferrées</h5>
    <ul style="list-style-type: none; padding-left: 0;">
"""
for t, color in railway_color_map.items():
    legend_html += f"""
        <li style="margin-bottom: 5px;">
            <span style="display: inline-block; width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"></span>
            {t}
        </li>
    """
legend_html += """
    </ul>
</div>
"""

# Ajouter la légende à la carte
m.get_root().html.add_child(folium.Element(legend_html))

# Sauvegarder la carte interactive au format HTML
m.save("mapvoiesferrees.html")
print("Carte interactive sauvegardée sous 'mapvoiesferrees.html'")

Colonnes dans le fichier 'voies_ferrees' : Index(['osm_id', 'code', 'fclass', 'name', 'layer', 'bridge', 'tunnel',
       'geometry'],
      dtype='object')


<ipython-input-3-fd2b96321357>:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_voies_ferrees.geometry.centroid.y.mean(),  # Use gdf_voies_ferrees for center
<ipython-input-3-fd2b96321357>:28: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_voies_ferrees.geometry.centroid.x.mean()


Carte interactive sauvegardée sous 'mapvoiesferrees.html'


**carte de batiments**

In [ ]:
import geopandas as gpd
import requests
import folium
from io import BytesIO

# URLs des fichiers GeoParquet

url_buildings = "https://github.com/Ghithub23/final/raw/main/fichier_random_1000_buildings.geo.parquet"

# Fonction pour charger un GeoParquet depuis une URL
def load_geoparquet(url):
    response = requests.get(url)
    response.raise_for_status()  # Vérifie que la réponse est valide
    return gpd.read_parquet(BytesIO(response.content))

# Charger les GeoParquets
gdf_buildings = load_geoparquet(url_buildings)

# Afficher les colonnes disponibles dans chaque GeoParquet
print("Colonnes dans le fichier 'buildings' :", gdf_buildings.columns)

# Mettre à jour les limites géographiques au Maroc
gdf_buildings = gdf_buildings.cx[-17.0:-1.0, 21.0:36.0]

# Créer une carte Folium centrée sur le Maroc
center_coords = [
    gdf_buildings.geometry.centroid.y.mean(),
    gdf_buildings.geometry.centroid.x.mean()
]
m = folium.Map(location=center_coords, zoom_start=6)

# Palette de couleurs pour les bâtiments
color_palette_buildings = [
    "red", "blue", "green", "yellow", "orange", "purple",
    "cyan", "magenta", "lime", "pink"
]
unique_building_types = gdf_buildings['type'].dropna().unique()
building_color_map = {t: color_palette_buildings[i % len(color_palette_buildings)] for i, t in enumerate(unique_building_types)}


# Ajouter les bâtiments avec des couleurs spécifiques et une bordure plus épaisse
for _, row in gdf_buildings.iterrows():
    color = building_color_map.get(row['type'], "gray")  # Gris par défaut si type inconnu
    folium.GeoJson(
        row.geometry.__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 3,  # Bordure plus épaisse
            'fillOpacity': 0.6
        }
    ).add_to(m)


# Créer une légende combinée pour les bâtiments et les voies ferrées
legend_html = """
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 250px; height: auto;
    z-index: 1000; background-color: white;
    padding: 10px; border: 2px solid black; border-radius: 5px;
">
    <h4>Légende</h4>
    <h5>Bâtiments</h5>
    <ul style="list-style-type: none; padding-left: 0;">
"""
for t, color in building_color_map.items():
    legend_html += f"""
        <li style="margin-bottom: 5px;">
            <span style="display: inline-block; width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"></span>
            {t}
        </li>
    """
</div>
"""

# Ajouter la légende à la carte
m.get_root().html.add_child(folium.Element(legend_html))

# Sauvegarder la carte interactive au format HTML
m.save("mapbuildings.html")
print("Carte interactive sauvegardée sous 'mapbuildings.html'")


**tri de la couche buildings en ne gardant que les bâtiments classifiés**

In [ ]:
import geopandas as gpd

# Charger le fichier GeoParquet
input_file = "/content/buildings_parq.parquet"
gdf = gpd.read_parquet(input_file)

# Filtrer les éléments dont la colonne 'type' est remplie
filtered_gdf = gdf[gdf['type'].notna() & (gdf['type'] != "")]

# Enregistrer le résultat dans un nouveau fichier GeoParquet
output_file = "/content//buildings_filtré.parquet"
filtered_gdf.to_parquet(output_file)

print(f"Fichier filtré enregistré sous : {output_file}")


Fichier filtré enregistré sous : /content//buildings_filtré.parquet


**carte de batiments triés**

In [ ]:
pip install geopandas folium requests pyarrow


In [ ]:
import geopandas as gpd

# Charger le fichier GeoParquet
gdf = gpd.read_parquet("buildings_filtré.parquet")

# Sélectionner 2000 éléments aléatoires
gdf_sampled = gdf.sample(n=2000, random_state=42) if len(gdf) > 2000 else gdf

# Sauvegarder le résultat
gdf_sampled.to_parquet("buildings_sampled.parquet", engine="pyarrow")

print("Échantillon sauvegardé dans 'buildings_sampled.parquet'.")


Échantillon sauvegardé dans 'buildings_sampled.parquet'.


In [ ]:
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
import folium
from io import BytesIO

# URL de votre fichier GeoParquet
url = "https://github.com/Ghithub23/final/raw/main/buildings_sampled.parquet"

# Étape 1 : Télécharger le fichier GeoParquet en mémoire
response = requests.get(url)
response.raise_for_status()  # Vérifie que la réponse est valide

# Charger dans un GeoDataFrame en utilisant BytesIO
gdf = gpd.read_parquet(BytesIO(response.content))

# Afficher les colonnes du GeoParquet
print("Colonnes disponibles dans le fichier GeoParquet :")
print(gdf.columns)

# Mettre à jour les limites géographiques au Maroc
gdf = gdf.cx[-17.0:-1.0, 21.0:36.0]

# Étape 3 : Visualisation interactive avec Folium
# Calcul des coordonnées centrales
center_coords = [
    gdf.geometry.centroid.y.mean(),
    gdf.geometry.centroid.x.mean()
]
m = folium.Map(location=center_coords, zoom_start=6)

# Ajouter les entités GeoJSON à la carte
for _, row in gdf.iterrows():
    # Access the __geo_interface__ attribute directly from the geometry object
    folium.GeoJson(row.geometry.__geo_interface__).add_to(m)

# Enregistrer la carte interactive au format HTML
m.save("geo_parquet_map_1.html")
print("Carte interactive sauvegardée sous 'geo_parquet_map_1.html'")

Colonnes disponibles dans le fichier GeoParquet :
Index(['osm_id', 'code', 'fclass', 'name', 'type', 'geometry'], dtype='object')


<ipython-input-20-31ff3fad754d>:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf.geometry.centroid.y.mean(),
<ipython-input-20-31ff3fad754d>:28: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf.geometry.centroid.x.mean()


Carte interactive sauvegardée sous 'geo_parquet_map_1.html'


**Amélioration de l'affichage de la légende**

In [ ]:
import geopandas as gpd
import requests
import folium
from io import BytesIO

# URL de votre fichier GeoParquet
url = "https://github.com/Ghithub23/final/raw/main/buildings_sampled.parquet"

# Étape 1 : Télécharger le fichier GeoParquet en mémoire
response = requests.get(url)
response.raise_for_status()  # Vérifie que la réponse est valide

# Charger dans un GeoDataFrame en utilisant BytesIO
gdf = gpd.read_parquet(BytesIO(response.content))

# Afficher les colonnes du GeoParquet
print("Colonnes disponibles dans le fichier GeoParquet :")
print(gdf.columns)

# Mettre à jour les limites géographiques au Maroc
gdf = gdf.cx[-17.0:-1.0, 21.0:36.0]

# Filtrer les types pour ne garder que les éléments mentionnés
valid_types = ['apartements', 'commercial', 'greenhouse', 'house', 'industrial', 'residential', 'roof', 'ruins', 'school']
gdf_filtered = gdf[gdf['type'].isin(valid_types)]

# Étape 2 : Créer une carte Folium
# Calcul des coordonnées centrales
center_coords = [
    gdf_filtered.geometry.centroid.y.mean(),
    gdf_filtered.geometry.centroid.x.mean()
]
m = folium.Map(location=center_coords, zoom_start=6)

# Palette de couleurs pour les types filtrés
color_palette = [
    "red", "brown", "green", "blue", "orange", "purple",
    "cyan", "magenta", "lime", "pink"
]

# Associer chaque type unique à une couleur de la palette
color_map = {t: color_palette[i % len(color_palette)] for i, t in enumerate(valid_types)}

# Étape 3 : Ajouter les entités GeoJSON à la carte avec couleurs
for _, row in gdf_filtered.iterrows():
    # Récupérer la couleur en fonction du type
    color = color_map.get(row['type'], "gray")  # Gris par défaut si type inconnu

    # Ajouter le GeoJSON avec une couleur personnalisée
    folium.GeoJson(
        row.geometry.__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 3,  # Augmenter l'épaisseur de la bordure
            'fillOpacity': 0.6
        }
    ).add_to(m)

# Étape 4 : Ajouter une légende en deux parties
legend_html = """
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 250px; height: auto;
    z-index: 1000; background-color: white;
    padding: 10px; border: 2px solid black; border-radius: 5px;
">
    <h4>Types de bâtiments (1/2)</h4>
    <ul style="list-style-type: none; padding-left: 0;">
"""

# Diviser la légende en deux parties
mid_index = len(color_map) // 2
first_half = list(color_map.items())[:mid_index]
second_half = list(color_map.items())[mid_index:]

# Ajouter les types dans la première moitié
for t, color in first_half:
    legend_html += f"""
        <li style="margin-bottom: 5px;">
            <span style="display: inline-block; width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"></span>
            {t}
        </li>
    """
legend_html += "</ul></div>"

# Ajouter la deuxième partie de la légende
legend_html_2 = """
<div style="
    position: fixed;
    bottom: 50px; left: 320px; width: 250px; height: auto;
    z-index: 1000; background-color: white;
    padding: 10px; border: 2px solid black; border-radius: 5px;
">
    <h4>Types de bâtiments (2/2)</h4>
    <ul style="list-style-type: none; padding-left: 0;">
"""
for t, color in second_half:
    legend_html_2 += f"""
        <li style="margin-bottom: 5px;">
            <span style="display: inline-block; width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"></span>
            {t}
        </li>
    """
legend_html_2 += "</ul></div>"

# Ajouter les légendes à la carte
m.get_root().html.add_child(folium.Element(legend_html))
m.get_root().html.add_child(folium.Element(legend_html_2))

# Étape 5 : Sauvegarder la carte interactive au format HTML
m.save("geo_parquet_map_filtered_with_split_legend.html")
print("Carte interactive sauvegardée sous 'geo_parquet_map_filtered_with_split_legend.html'")


Colonnes disponibles dans le fichier GeoParquet :
Index(['osm_id', 'code', 'fclass', 'name', 'type', 'geometry'], dtype='object')


<ipython-input-27-c1a43503d6ba>:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_filtered.geometry.centroid.y.mean(),
<ipython-input-27-c1a43503d6ba>:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_filtered.geometry.centroid.x.mean()


Carte interactive sauvegardée sous 'geo_parquet_map_filtered_with_split_legend.html'
